In [1]:
import sys
import os
sys.path.append(os.path.dirname(sys.path[0])) # append project directory to path for import
os.chdir(os.path.dirname(sys.path[0])) # change current working directory to project directory

%load_ext autoreload
%autoreload 2

In [2]:
import processor
from processor.accumulator import LumiAccumulator

In [3]:
l1 = LumiAccumulator([1, 1], [2, 3], auto_unique=False)
l2 = LumiAccumulator([1, 1, 8], [3, 4, 9])

In [4]:
l3 = (l1 + l2)
#l3.unique()
l3.array

array([[1, 2],
       [1, 3],
       [1, 3],
       [1, 4],
       [8, 9]])

In [5]:
out1 = {"lumi": {"JetMET": l1}}
out2 = {"lumi": {"JetMET": l2}}

In [6]:
l1 + l2

In [7]:
import numpy as np
import coffea
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory
from coffea.lumi_tools import LumiList

In [8]:
out_mc = coffea.util.load("coffea_output/AK4Puppi_HLT_comparison_correction_mc_test10_testconfig_testlumilist.coffea")
print(out_mc["process_time"])

0:04:56.126493


In [9]:
out_mc_lumiacc = coffea.util.load("coffea_output/AK4Puppi_HLT_comparison_correction_mc_test10_testconfig_testlumiacc.coffea")
print(out_mc_lumiacc["process_time"])

0:04:56.109424


In [10]:
fname = "/data/data/JME_NANO_DATA/2022/JMENanoRun3_v2p1_Run2022D-PromptReco-v2/JetMET/220915_173253/0000/tree_1.root"
events = NanoEventsFactory.from_root(
            fname, 
            schemaclass=NanoAODSchema,
            metadata={"dataset": "test"}
            ).events()

In [11]:
%%timeit
lumi_list = list(set(zip(events.run, events.luminosityBlock)))

101 ms ± 409 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
lumi_acc = LumiAccumulator(events.run, events.luminosityBlock, auto_unique=True)

22.1 ms ± 463 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
lumi_list_sum = list(set(zip(events.run, events.luminosityBlock)))
for i in range(10):
    lumi_list_sum += list(set(zip(events.run, events.luminosityBlock)))
lumi_list_sum = np.array(lumi_list_sum)
lumi_list_sum = LumiList(lumi_list_sum[:, 0], lumi_list_sum[:, 1])

1.11 s ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
lumi_list_sum = list(zip(events.run, events.luminosityBlock))
for i in range(10):
    lumi_list_sum += list(zip(events.run, events.luminosityBlock))
lumi_list_sum = np.array(lumi_list_sum)
lumi_list_sum = LumiList(lumi_list_sum[:, 0], lumi_list_sum[:, 1])

1.61 s ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
lumi_list_run = list(events.run)
lumi_list_lumiblock = list(events.luminosityBlock)
for i in range(10):
    lumi_list_run += list(events.run)
    lumi_list_lumiblock += list(events.luminosityBlock)
lumi_list_sum = LumiList(lumi_list_run, lumi_list_lumiblock)

1.44 s ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
lumi_acc_sum = LumiAccumulator(events.run, events.luminosityBlock, auto_unique=True)
for i in range(10):
    lumi_acc_sum += LumiAccumulator(events.run, events.luminosityBlock, auto_unique=True)
lumi_acc_sum.unique()

245 ms ± 7.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
lumi_acc_sum = LumiAccumulator(events.run, events.luminosityBlock, auto_unique=False)
for i in range(10):
    lumi_acc_sum += LumiAccumulator(events.run, events.luminosityBlock, auto_unique=False)
lumi_acc_sum.unique()

239 ms ± 949 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
